 # Instalacja potrzebnych bibliotek jak nie są zainstalowane. Rekomendowane użycie wirtualnego środowiska.

In [48]:
import subprocess
import sys

packages = ["pandas", "scikit-learn", "tqdm", "numpy"]

for package in packages:
    try:
        __import__(package)
        print(f"{package} jest już zainstalowany.")
    except ImportError:
        print(f"{package} nie jest zainstalowany. Instalowanie...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", package])

pandas jest już zainstalowany.
scikit-learn nie jest zainstalowany. Instalowanie...
tqdm jest już zainstalowany.
numpy jest już zainstalowany.


 # Import potrzebnych bibliotek

In [49]:
import os
import pandas as pd
import pickle
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from tqdm import tqdm  # Dodajemy tqdm do monitorowania postępu
import numpy as np

 # Ustawianie ścieżek plików

In [50]:
current_dir = os.getcwd()
model_path = os.path.join(current_dir, 'Model/BRANSON.model')
data_path = os.path.join(current_dir, 'Data')
test_data_path = os.path.join(current_dir, 'TestData')

 # Parametry modelu:
- n_estimators - Liczba drzew w Random Forest
- test_size - Procent danych do testowania
- random_state - Dla powtarzalności wyników

In [51]:
n_estimators = 100
test_size = 0.3
random_state = 42

 # Tworzenie tabeli x,y

In [52]:
X = np.arange(5000)
y = np.arange(5000)

 # Sprawdzenie, czy istnieje zapisany model

In [53]:
model_exists = os.path.exists(model_path)

 # Funkcja do wyświetlania menu z kolorowaniem tekstu

In [54]:
def display_menu():
    print("\nWybierz akcję:")
    print("a) Generuj nowy model")
    if model_exists:
        print("\033[92mb) Doucz istniejący model\033[0m")  # Zielony tekst
        print("\033[92mc) Użyj istniejącego modelu do analizy wyników\033[0m")  # Zielony tekst
    else:
        print("\033[90mb) Doucz istniejący model (niedostępne, brak modelu)\033[0m")  # Szary tekst
        print("\033[90mc) Użyj istniejącego modelu do analizy wyników (niedostępne, brak modelu)\033[0m")  # Szary tekst

 # Funkcja do sprawdzania unikalnych wartości w kolumnach

In [55]:
def check_column_types(data):
    for column in data.columns:
        unique_values = data[column].unique()
        print(f"Kolumna: {column}, Unikalne wartości: {unique_values}")

 # Funkcja do konwersji wartości tekstowych na liczby

In [56]:
def convert_values_to_numeric(data):
    for column in data.columns:
        if data[column].dtype == 'object':
            print(f"Konwertuję wartości w kolumnie: {column}")
            unique_values = data[column].unique()
            print(f"Unikalne wartości: {unique_values}")
            # Konwersja na liczby dla każdej kolumny z danymi tekstowymi
            data[column] = data[column].apply(lambda x: 1 if x in ['OK', 'flat'] else 
                                                (0 if x in ['not flat', 'Leak Detected', 'Maintenance Needed'] else x))
    return data

# Funkcja do generowania nowego modelu

In [57]:
def generate_new_model():
    print("\nGenerowanie nowego modelu...\n")
    
    # Załaduj dane z plików CSV
    print("Ładowanie danych...")
    injection_data = pd.read_csv(os.path.join(data_path, 'injection_traceability.csv'))
    robot_data = pd.read_csv(os.path.join(data_path, 'robot_traceability.csv'))

    # Sprawdź kolumny robot_data i wyświetl unikalne wartości
    check_column_types(robot_data)

    # Konwersja wartości tekstowych na liczby
    print("Konwersja danych na liczby...")
    robot_data = convert_values_to_numeric(robot_data)
    injection_data = convert_values_to_numeric(injection_data)

    # Połączenie danych z wtryskarki i robota na podstawie Cycle ID
    print("Łączenie danych z wtryskarki i robota...")
    data = pd.merge(injection_data, robot_data, on="Cycle ID")

    # Przygotowanie zmiennych X (cechy) i y (target - czy element upadł)
    X = data.drop(columns=["Cycle ID", "Element Dropped"])  # Wszystkie kolumny poza Cycle ID i targetem
    y = data["Element Dropped"]  # Target - czy element upadł (True/False)

    # Podział na dane treningowe i testowe
    print("Podział na dane treningowe i testowe...")
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=random_state)

    # Stworzenie modelu Random Forest z włączoną wielowątkowością (n_jobs=-1)
    print("Tworzenie modelu Random Forest...")
    model = RandomForestClassifier(n_estimators=n_estimators, random_state=random_state, n_jobs=-1)

    # Trenowanie modelu z monitorowaniem postępu
    print("Rozpoczęcie trenowania modelu...")
    for i in tqdm(range(n_estimators)):
        model.fit(X_train, y_train)

    # Zapis modelu do pliku
    print("Zapis modelu do pliku...")
    with open(model_path, 'wb') as file:
        pickle.dump(model, file)
    
    print("Nowy model został wygenerowany i zapisany jako 'BRANSON_AI_MODEL'.")
    print("Raport z klasyfikacji dla danych testowych:\n")
    y_pred = model.predict(X_test)
    print(classification_report(y_test, y_pred))

# Funkcja do douczania istniejącego modelu

In [58]:
def retrain_existing_model():
    print("\nDouczanie istniejącego modelu...\n")
    
    # Załaduj dane z plików CSV
    injection_data = pd.read_csv(os.path.join(data_path, 'injection_traceability.csv'))
    robot_data = pd.read_csv(os.path.join(data_path, 'robot_traceability.csv'))

    # Konwersja wartości tekstowych na liczby
    robot_data = convert_values_to_numeric(robot_data)
    injection_data = convert_values_to_numeric(injection_data)  # Konwersja także dla wtryskarki

    # Połączenie danych z wtryskarki i robota na podstawie Cycle ID
    data = pd.merge(injection_data, robot_data, on="Cycle ID")

    # Przygotowanie zmiennych X (cechy) i y (target - czy element upadł)
    X = data.drop(columns=["Cycle ID", "Element Dropped"])  # Wszystkie kolumny poza Cycle ID i targetem
    y = data["Element Dropped"]  # Target - czy element upadł (True/False)

    # Podział na dane treningowe i testowe
    # X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=random_state) - nie używamy tego!!!

    split_index = int(0.2 * len(X))

    # podział danych

    X_train, X_test = X[:split_index], X[split_index:]
    y_train, y_test = y[:split_index], y[split_index:]

    # Załaduj istniejący model
    with open(model_path, 'rb') as file:
        model = pickle.load(file)

    # Dalsze trenowanie modelu
    model.fit(X_train, y_train)

    # Nadpisanie istniejącego modelu
    with open(model_path, 'wb') as file:
        pickle.dump(model, file)
    
    print("Model został douczony i zapisany ponownie.")
    print("Raport z klasyfikacji dla danych testowych:\n")
    y_pred = model.predict(X_test)
    print(classification_report(y_test, y_pred))

# Funkcja do używania istniejącego modelu do analizy nowych danych

In [59]:
def use_existing_model():
    print("\nUżywanie istniejącego modelu do analizy wyników...\n")

    # Załaduj dane testowe (np. nowe dane z testów produkcji)
    test_data = pd.read_csv(os.path.join(test_data_path, 'test_data.csv'))

    # Załaduj istniejący model
    with open(model_path, 'rb') as file:
        model = pickle.load(file)

    # Przewidywanie na danych testowych
    X_test = test_data.drop(columns=["Cycle ID", "Element Dropped"])  # Wszystkie kolumny poza Cycle ID i targetem
    y_test = test_data["Element Dropped"]  # Target - czy element upadł (True/False)

    y_pred = model.predict(X_test)

    # Wyświetlanie raportu z klasyfikacji
    print("Raport z klasyfikacji dla nowych danych:\n")
    print(classification_report(y_test, y_pred))

# Wyświetlenie menu
a) Generuj nowy model

b) Doucz istniejący model (niedostępne, brak modelu)

c) Użyj istniejącego modelu do analizy wyników (niedostępne, brak modelu)

In [60]:
display_menu()

# Wybór użytkownika
choice = input("\nWybierz opcję (a, b, c): ").lower()

# Obsługa wyboru użytkownika
if choice == 'a':
    generate_new_model()
elif choice == 'b':
    if model_exists:
        retrain_existing_model()
    else:
        print("\nOpcja douczenia modelu jest niedostępna, ponieważ nie ma istniejącego modelu.")
elif choice == 'c':
    if model_exists:
        use_existing_model()
    else:
        print("\nOpcja analizy modelu jest niedostępna, ponieważ nie ma istniejącego modelu.")
else:
    print("\nNieprawidłowy wybór. Proszę wybrać 'a', 'b' lub 'c'.")


Wybierz akcję:
a) Generuj nowy model
b) Doucz istniejący model (niedostępne, brak modelu)
c) Użyj istniejącego modelu do analizy wyników (niedostępne, brak modelu)

Generowanie nowego modelu...

Ładowanie danych...
Kolumna: Cycle ID, Unikalne wartości: [    1     2     3 ... 19998 19999 20000]
Kolumna: Suction Power, Unikalne wartości: [78.70874056 85.03580208 73.51615297 ... 84.71737665 74.5475385
 81.23827799]
Kolumna: Positioning Error, Unikalne wartości: [ 0.00963573  0.007319   -0.04213619 ...  0.03415695 -0.00121158
 -0.00793526]
Kolumna: Vacuum Pressure, Unikalne wartości: [97.90518417 90.9188059  95.69204835 ... 87.81796798 92.55350259
 97.95315697]
Kolumna: Speed, Unikalne wartości: [2.56982101 2.60551191 2.43373355 ... 2.44616698 2.32317105 2.82291761]
Kolumna: Tilt Angle, Unikalne wartości: [-0.52443661 -0.47441019 -0.2985445  ... -0.9186953   0.32664896
 -0.49569715]
Kolumna: Suction Force, Unikalne wartości: [74.31854381 83.48102116 84.82523999 ... 84.06032228 83.33495628

100%|██████████| 100/100 [04:02<00:00,  2.43s/it]

Zapis modelu do pliku...
Nowy model został wygenerowany i zapisany jako 'BRANSON_AI_MODEL'.
Raport z klasyfikacji dla danych testowych:

              precision    recall  f1-score   support

       False       0.98      1.00      0.99      3651
        True       1.00      0.97      0.98      2349

    accuracy                           0.99      6000
   macro avg       0.99      0.98      0.99      6000
weighted avg       0.99      0.99      0.99      6000

